In [37]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = '2.air_quality_data_with_impurities_stage2.csv'
df = pd.read_csv(file_path)


In [38]:
# Display basic information about the dataset
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Year                           244 non-null    object 
 1   Country                        244 non-null    object 
 2   Air Pollutant                  244 non-null    object 
 3   Population                     244 non-null    int64  
 4   Populated Area [km2]           244 non-null    object 
 5   Air Pollution Average [ug/m3]  244 non-null    object 
 6   Premature Deaths               186 non-null    float64
 7   Years Of Life Lost             186 non-null    float64
dtypes: float64(2), int64(1), object(5)
memory usage: 15.4+ KB
None
   Year  Country Air Pollutant  Population Populated Area [km2]  \
0  2018  Austria           NO2     8822183                43050   
1  2018  Austria            O3     8822183             43050 km   
2  2018  Austria          P

In [39]:
# Detect and Remove Duplicates 

In [40]:
# Check for Duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")


Number of duplicate rows: 37


In [41]:
# Remove Duplicates
def remove_duplicates(df):
    """
    Removes duplicate rows from the DataFrame and confirms their removal.
    """
    # Remove duplicate rows from the DataFrame
    df.drop_duplicates(inplace=True)
    
    # Confirm that duplicates have been removed
    if df.duplicated().sum() == 0:
        print("No duplicate rows present.")
    else:
        print("Warning: Duplicate rows still present!")

    return df

# Applying the function to the DataFrame
df = remove_duplicates(df)


No duplicate rows present.


In [42]:
# Detect and Handle Missing Values

In [43]:
# Check for Missing Values
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)


Missing values per column:
Year                              0
Country                           0
Air Pollutant                     0
Population                        0
Populated Area [km2]              0
Air Pollution Average [ug/m3]     0
Premature Deaths                 52
Years Of Life Lost               52
dtype: int64


In [44]:
# Impute Missing values
def clean_missing_values(df):
    """
    Imputes missing values in specified columns of the DataFrame with the mean of those columns after converting the column to correct data type.
    """
    # Impute missing values in 'Premature Deaths' with the mean of the column
    df['Premature Deaths'] = df['Premature Deaths'].astype(float).fillna(df['Premature Deaths'].mean())

    # Impute missing values in 'Years Of Life Lost' with the mean of the column
    df['Years Of Life Lost'] = df['Years Of Life Lost'].astype(float).fillna(df['Years Of Life Lost'].mean())
    
    return df

# Apply the cleaning function to the DataFrame
df_cleaned = clean_missing_values(df)

# Display the number of missing values after cleaning to confirm
missing_values_after_cleaning = df_cleaned.isnull().sum()
print(missing_values_after_cleaning)


Year                             0
Country                          0
Air Pollutant                    0
Population                       0
Populated Area [km2]             0
Air Pollution Average [ug/m3]    0
Premature Deaths                 0
Years Of Life Lost               0
dtype: int64


In [45]:
# Check and standarize Inconsistent Formats in Year column

In [46]:
# Check for Inconsistent Formats in 'Year' column
if 'Year' in df.columns:
    inconsistent_year = df['Year'].apply(lambda x: not str(x).isdigit()).sum()
    print(f"Inconsistent formats in 'Year' column: {inconsistent_year}")



Inconsistent formats in 'Year' column: 10


In [47]:
# Standarize Year Format
def standardize_year_format(df):
    """
    Standardizes the format of the 'Year' column in the DataFrame.
    Removes any non-numeric characters and converts the column to integer type.
    """
    # Convert 'Year' to string, remove any trailing text, and then convert back to integer
    df['Year'] = df['Year'].astype(str).str.replace(' year', '').astype(int)
    return df

# Apply the standardization function to the DataFrame
df = standardize_year_format(df)

# Optionally, display the DataFrame to verify the changes
print(df.head())


   Year  Country Air Pollutant  Population Populated Area [km2]  \
0  2018  Austria           NO2     8822183                43050   
1  2018  Austria            O3     8822183             43050 km   
2  2018  Austria          PM10     8822183                43050   
3  2018  Austria         PM2.5     8822183                43050   
4  2019  Austria           NO2     8858695                43050   

  Air Pollution Average [ug/m3]  Premature Deaths  Years Of Life Lost  
0                           9.5       1228.000000        12796.000000  
1                        7867.9        619.000000         6756.000000  
2                          15.4       8572.548387        94127.948387  
3                          11.2       5001.000000        52145.000000  
4                           8.7       1038.000000        10735.000000  


In [48]:
# Standardize Text and Categorical Data

In [49]:
def ensure_countries_start_uppercase(df):
    """
    Ensures that country names in the 'Country' column start with an uppercase letter.
    Converts them to title case if they don't.
    """
    # Convert country names to title case if they don't start with an uppercase letter
    df['Country'] = df['Country'].apply(lambda x: x.title() if isinstance(x, str) and not x.istitle() else x)

    return df

# Applying the function to the DataFrame
df = ensure_countries_start_uppercase(df)

# Optionally, display the DataFrame to verify the changes
print("Country names after ensuring they start with an uppercase letter:")
print(sorted(df['Country'].unique()))

Country names after ensuring they start with an uppercase letter:
['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'European Union Countries', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Slovakia', 'Slovenia', 'Spain']


In [50]:
# Check for Non-Standardized Entries in Country Column 
def check_unstandardized_countries(df, country_pairs):
    """
    Checks for unstandardized country names in the 'Country' column of the DataFrame
    based on a provided dictionary of standardized country names.
    """
    unstandardized_countries = df['Country'].apply(lambda x: x not in country_pairs.values()).sum()
    return unstandardized_countries

# Dictionary to map country names from one naming convention to another.
# It will particularly useful when merging the datasets together.
country_pairs = {
    "Austria": "Austria",
    "Croatia": "Croatia",
    "Cyprus": "Cyprus",
    "Czechia": "Czech Republic",
    "Denmark": "Denmark",
    "Estonia": "Estonia",
    "European Union Countries": "European Union",
    "Finland": "Finland",
    "France": "France",
    "Germany": "Germany",
    "Greece": "Greece",
    "Hungary": "Hungary",
    "Ireland": "Ireland",
    "Italy": "Italy",
    "Latvia": "Latvia",
    "Lithuania": "Lithuania",
    "Luxembourg": "Luxembourg",
    "Malta": "Malta",
    "Netherlands": "Netherlands",
    "Poland": "Poland",
    "Portugal": "Portugal",
    "Romania": "Romania",
    "Slovakia": "Slovakia",
    "Slovenia": "Slovenia",
    "Spain": "Spain",
    "Sweden": "Sweden"
}

# Applying the function to check for unstandardized country names
num_unstandardized_countries = check_unstandardized_countries(df, country_pairs)
print(f"Unstandardized country names: {num_unstandardized_countries}")

Unstandardized country names: 37


In [51]:
def standardize_country_names(df, country_pairs):
    """
    Standardizes the country names in the DataFrame using a provided dictionary mapping.
    """
    # Replace country names using the dictionary
    df['Country'] = df['Country'].apply(lambda x: country_pairs.get(x, x))

    return df

# Applying the function to standardize country names
df = standardize_country_names(df, country_pairs)

#  Verify the changes
print("Country column after standardization:")
print(sorted(df['Country'].unique()))

Country column after standardization:
['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'European Union', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Slovakia', 'Slovenia', 'Spain']


In [52]:
# Detect and Clean Non-Numeric Characters in Numeric Columns

In [53]:
def detect_non_numeric_in_numeric_columns(df):
    """
    Detects non-numeric characters in columns that are intended to be numeric but might be incorrectly typed as strings.
    """
    impure_columns = []

    # We consider all columns for checking as some numeric columns may be incorrectly typed as object due to impurities
    for col in df.columns:
        # Detect non-numeric patterns in string representations of the column
        if df[col].apply(lambda x: isinstance(x, str) and not all(char.isdigit() or char in '.-' for char in x)).any():
            impure_columns.append(col)

    return impure_columns

# Detecting non-numeric characters in numeric columns
impure_numeric_columns = detect_non_numeric_in_numeric_columns(df)
print("Columns with non-numeric characters:", impure_numeric_columns)


Columns with non-numeric characters: ['Country', 'Air Pollutant', 'Populated Area [km2]', 'Air Pollution Average [ug/m3]']


In [54]:
def clean_numeric_columns(df):
    """
    Cleans columns in the DataFrame by removing non-numeric characters from columns that are intended to be numeric.
    """
    # Consider all columns for cleaning as some numeric columns may be incorrectly typed as object
    for col in df.columns:
        # Exclude specific columns from cleaning
        if col not in ['Country', 'Air Pollutant']:
            # Detect if the column contains non-numeric data (like letters or special characters)
            if df[col].apply(lambda x: isinstance(x, str) and not all(char.isdigit() or char in '.-' for char in x)).any():
                print(f"Cleaning non-numeric characters found in {col}")
                # Replace non-numeric characters with an empty string and convert to float
                df[col] = pd.to_numeric(df[col].astype(str).replace(r'[^0-9.\-]', '', regex=True), errors='coerce')

    return df

# Applying the cleaning function to the DataFrame
df_cleaned = clean_numeric_columns(df)

# Display the DataFrame to verify the changes
print("DataFrame after cleaning numeric columns:")
print(df_cleaned.head())


Cleaning non-numeric characters found in Populated Area [km2]
Cleaning non-numeric characters found in Air Pollution Average [ug/m3]
DataFrame after cleaning numeric columns:
   Year  Country Air Pollutant  Population  Populated Area [km2]  \
0  2018  Austria           NO2     8822183                 43050   
1  2018  Austria            O3     8822183                 43050   
2  2018  Austria          PM10     8822183                 43050   
3  2018  Austria         PM2.5     8822183                 43050   
4  2019  Austria           NO2     8858695                 43050   

   Air Pollution Average [ug/m3]  Premature Deaths  Years Of Life Lost  
0                            9.5       1228.000000        12796.000000  
1                         7867.9        619.000000         6756.000000  
2                           15.4       8572.548387        94127.948387  
3                           11.2       5001.000000        52145.000000  
4                            8.7       1038.000000 

In [55]:
# Check and Standardize the remaining Data Types

In [56]:
# Check for Incorrect Data Types
incorrect_data_types = df.dtypes
print("Data Types in all columns:")
print(incorrect_data_types)


Data Types in all columns:
Year                               int64
Country                           object
Air Pollutant                     object
Population                         int64
Populated Area [km2]               int64
Air Pollution Average [ug/m3]    float64
Premature Deaths                 float64
Years Of Life Lost               float64
dtype: object


In [57]:
def convert_all_column_data_types_with_error_handling(df):
    """
    Converts the data types of all columns in the DataFrame to their correct types,
    with error handling for unexpected or inconsistent data.
    """
    # Convert 'Year' to int
    try:
        df['Year'] = df['Year'].astype('int64')
    except ValueError:
        print("Error converting 'Year' to int. Check for non-integer values.")

    # Convert 'Country' to string (assuming it's categorical)
    df['Country'] = df['Country'].astype('object')

    # Convert 'Air Pollutant' to string (assuming it's categorical)
    df['Air Pollutant'] = df['Air Pollutant'].astype('object')

    # Convert 'Population' to int
    try:
        df['Population'] = df['Population'].astype('int64')
    except ValueError:
        print("Error converting 'Population' to int. Check for non-integer values.")

    # Convert 'Populated Area [km2]' to float
    try:
        df['Populated Area [km2]'] = pd.to_numeric(df['Populated Area [km2]'], errors='coerce')
    except ValueError:
        print("Error converting 'Populated Area [km2]' to float. Check for non-numeric characters.")

    # Convert 'Air Pollution Average [ug/m3]' to float
    try:
        df['Air Pollution Average [ug/m3]'] = pd.to_numeric(df['Air Pollution Average [ug/m3]'], errors='coerce')
    except ValueError:
        print("Error converting 'Air Pollution Average [ug/m3]' to float. Check for non-numeric characters.")

    # Convert 'Premature Deaths' to float
    try:
        df['Premature Deaths'] = df['Premature Deaths'].astype('float64')
    except ValueError:
        print("Error converting 'Premature Deaths' to float. Check for non-numeric characters.")

    # Convert 'Years Of Life Lost' to float
    try:
        df['Years Of Life Lost'] = df['Years Of Life Lost'].astype('float64')
    except ValueError:
        print("Error converting 'Years Of Life Lost' to float. Check for non-numeric characters.")

    return df

# Applying the function to the DataFrame
df = convert_all_column_data_types_with_error_handling(df)

# Display the DataFrame to verify the changes
print("Updated Data Types in DataFrame:")
print(df.dtypes)


Updated Data Types in DataFrame:
Year                               int64
Country                           object
Air Pollutant                     object
Population                         int64
Populated Area [km2]               int64
Air Pollution Average [ug/m3]    float64
Premature Deaths                 float64
Years Of Life Lost               float64
dtype: object


In [58]:
# Final Validation

In [59]:
print(df.info())
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
Index: 207 entries, 0 to 243
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Year                           207 non-null    int64  
 1   Country                        207 non-null    object 
 2   Air Pollutant                  207 non-null    object 
 3   Population                     207 non-null    int64  
 4   Populated Area [km2]           207 non-null    int64  
 5   Air Pollution Average [ug/m3]  207 non-null    float64
 6   Premature Deaths               207 non-null    float64
 7   Years Of Life Lost             207 non-null    float64
dtypes: float64(3), int64(3), object(2)
memory usage: 14.6+ KB
None
              Year    Population  Populated Area [km2]  \
count   207.000000  2.070000e+02          2.070000e+02   
mean   2018.487923  3.448848e+07          1.368710e+05   
std       0.501066  8.978549e+07          3.632800e+05   
m

In [60]:
# Check for Missing Values
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)

Missing values per column:
Year                             0
Country                          0
Air Pollutant                    0
Population                       0
Populated Area [km2]             0
Air Pollution Average [ug/m3]    0
Premature Deaths                 0
Years Of Life Lost               0
dtype: int64


In [61]:
# Adding Aggregation

In [62]:
# Add AQI column for each pollutant 
# Define the hypothetical annual AQI breakpoints for PM2.5, PM10, O3, NO2

In [63]:
#  AQI breakpoints for PM2.5 (in µg/m³)
aqi_breakpoints_pm25 = {
    (0.0, 12.0): (0, 50),
    (12.1, 35.4): (51, 100),
    (35.5, 55.4): (101, 150),
    (55.5, 150.4): (151, 200),
    (150.5, 250.4): (201, 300),
    (250.5, 500.4): (301, 500)
}

# AQI breakpoints for PM10 (in µg/m³)
aqi_breakpoints_pm10 = {
    (0, 54): (0, 50),
    (55, 154): (51, 100),
    (155, 254): (101, 150),
    (255, 354): (151, 200),
    (355, 424): (201, 300),
    (425, 604): (301, 500)
}

#  AQI breakpoints for O3 (in ppm for 8-hour average)
aqi_breakpoints_o3 = {
    (0.000, 0.054): (0, 50),
    (0.055, 0.070): (51, 100),
    (0.071, 0.085): (101, 150),
    (0.086, 0.105): (151, 200),
    (0.106, 0.200): (201, 300)
}

#  AQI breakpoints for NO2 (in ppb for annual average)
aqi_breakpoints_no2 = {
    (0, 53): (0, 50),
    (54, 100): (51, 100),
    (101, 360): (101, 150),
    (361, 649): (151, 200),
    (650, 1249): (201, 300)
}


In [64]:
# Function to calculate AQI
def calculate_aqi(concentration, breakpoints):
    for (low, high), (aqi_low, aqi_high) in breakpoints.items():
        if low <= concentration <= high:
            aqi = ((aqi_high - aqi_low) / (high - low)) * (concentration - low) + aqi_low
            return round(aqi, 2)
    return 0

# Function to determine AQI for each row
def row_to_aqi(row):
    pollutant = row['Air Pollutant']
    concentration = row['Air Pollution Average [ug/m3]']
    if pollutant == 'PM2.5':
        return calculate_aqi(concentration, aqi_breakpoints_pm25)
    elif pollutant == 'PM10':
        return calculate_aqi(concentration, aqi_breakpoints_pm10)
    elif pollutant == 'O3':
        # Convert O3 concentration to ppm 
        concentration_ppm = concentration / 1000  # Assuming data is in µg/m³
        return calculate_aqi(concentration_ppm, aqi_breakpoints_o3)
    elif pollutant == 'NO2':
        # Convert NO2 concentration to ppb 
        concentration_ppb = concentration  # Assuming data is already in ppb
        return calculate_aqi(concentration_ppb, aqi_breakpoints_no2)
    return None

# Apply the function to each row in the DataFrame
df['Annual AQI'] = df.apply(row_to_aqi, axis=1)

# Display the first few rows to check the results
print(df.head())

   Year  Country Air Pollutant  Population  Populated Area [km2]  \
0  2018  Austria           NO2     8822183                 43050   
1  2018  Austria            O3     8822183                 43050   
2  2018  Austria          PM10     8822183                 43050   
3  2018  Austria         PM2.5     8822183                 43050   
4  2019  Austria           NO2     8858695                 43050   

   Air Pollution Average [ug/m3]  Premature Deaths  Years Of Life Lost  \
0                            9.5       1228.000000        12796.000000   
1                         7867.9        619.000000         6756.000000   
2                           15.4       8572.548387        94127.948387   
3                           11.2       5001.000000        52145.000000   
4                            8.7       1038.000000        10735.000000   

   Annual AQI  
0        8.96  
1        0.00  
2       14.26  
3       46.67  
4        8.21  


In [65]:
# Check again after aggregation for missing values and duplicates

In [66]:
# Check for Missing Values
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)

# Check for Duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")


Missing values per column:
Year                             0
Country                          0
Air Pollutant                    0
Population                       0
Populated Area [km2]             0
Air Pollution Average [ug/m3]    0
Premature Deaths                 0
Years Of Life Lost               0
Annual AQI                       0
dtype: int64
Number of duplicate rows: 13


In [67]:
df = df.drop_duplicates()

In [68]:
# Pivot the data to get all values in separate columns for each Air Pollutant
aqi_full_pivot = df.pivot_table(index=['Year', 'Country', 'Population', 'Populated Area [km2]'], 
                                columns='Air Pollutant', 
                                values=['Air Pollution Average [ug/m3]',
                                        'Premature Deaths',
                                        'Years Of Life Lost',
                                        'Annual AQI'],
                                aggfunc='first').reset_index()

# Flattening the multi-level column headers
aqi_full_pivot.columns = [' '.join(col).strip() for col in aqi_full_pivot.columns.values]

# Display the transformed dataframe
aqi_full_pivot.head()

,Year,Country,Population,Populated Area [km2],Air Pollution Average [ug/m3] NO2,Air Pollution Average [ug/m3] O3,Air Pollution Average [ug/m3] PM10,Air Pollution Average [ug/m3] PM2.5,Annual AQI NO2,Annual AQI O3,Annual AQI PM10,Annual AQI PM2.5,Premature Deaths NO2,Premature Deaths O3,Premature Deaths PM10,Premature Deaths PM2.5,Years Of Life Lost NO2,Years Of Life Lost O3,Years Of Life Lost PM10,Years Of Life Lost PM2.5
0,2018,Austria,8822183,43050,9.5,7867.9,15.4,11.2,8.96,0.0,14.26,46.67,1228.0,619.0,8572.548387,5001.0,12796.0,6756.0,94127.948387,52145.0
1,2018,Belgium,11398472,25153,14.1,4803.8,18.5,11.5,13.30,0.0,17.13,47.92,2085.0,508.0,8572.548387,5914.0,22293.0,5810.0,94127.948387,62914.0
2,2018,Bulgaria,7049950,23519,9.3,6089.9,20.8,14.7,8.77,0.0,19.26,56.47,1883.0,467.0,8572.548387,12027.0,20285.0,5239.0,94127.948387,129430.0
3,2018,Croatia,4105440,23884,6.4,6583.1,18.6,13.0,6.04,0.0,17.22,52.89,462.0,369.0,8572.548387,4696.0,4778.0,3945.0,94127.948387,48568.0
4,2018,Cyprus,1197024,6504,7.8,9265.6,26.4,12.6,7.36,0.0,24.44,52.05,201.0,59.0,8572.548387,526.0,2215.0,687.0,94127.948387,5769.0


In [69]:
# Identify the columns to round
columns_to_round = [col for col in aqi_full_pivot.columns if 'Air Pollution Average [ug/m3]' in col 
                    or 'Premature Deaths' in col 
                    or 'Years Of Life Lost' in col 
                    or 'Annual AQI' in col]

# Round these columns to 2 decimal places
aqi_full_pivot[columns_to_round] = aqi_full_pivot[columns_to_round].round(2)

# Display the transformed dataframe
aqi_full_pivot.head()


,Year,Country,Population,Populated Area [km2],Air Pollution Average [ug/m3] NO2,Air Pollution Average [ug/m3] O3,Air Pollution Average [ug/m3] PM10,Air Pollution Average [ug/m3] PM2.5,Annual AQI NO2,Annual AQI O3,Annual AQI PM10,Annual AQI PM2.5,Premature Deaths NO2,Premature Deaths O3,Premature Deaths PM10,Premature Deaths PM2.5,Years Of Life Lost NO2,Years Of Life Lost O3,Years Of Life Lost PM10,Years Of Life Lost PM2.5
0,2018,Austria,8822183,43050,9.5,7867.9,15.4,11.2,8.96,0.0,14.26,46.67,1228.0,619.0,8572.55,5001.0,12796.0,6756.0,94127.95,52145.0
1,2018,Belgium,11398472,25153,14.1,4803.8,18.5,11.5,13.30,0.0,17.13,47.92,2085.0,508.0,8572.55,5914.0,22293.0,5810.0,94127.95,62914.0
2,2018,Bulgaria,7049950,23519,9.3,6089.9,20.8,14.7,8.77,0.0,19.26,56.47,1883.0,467.0,8572.55,12027.0,20285.0,5239.0,94127.95,129430.0
3,2018,Croatia,4105440,23884,6.4,6583.1,18.6,13.0,6.04,0.0,17.22,52.89,462.0,369.0,8572.55,4696.0,4778.0,3945.0,94127.95,48568.0
4,2018,Cyprus,1197024,6504,7.8,9265.6,26.4,12.6,7.36,0.0,24.44,52.05,201.0,59.0,8572.55,526.0,2215.0,687.0,94127.95,5769.0


In [70]:
# Final check 

In [71]:
# Check for Missing Values
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)

# Check for Duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")


Missing values per column:
Year                             0
Country                          0
Air Pollutant                    0
Population                       0
Populated Area [km2]             0
Air Pollution Average [ug/m3]    0
Premature Deaths                 0
Years Of Life Lost               0
Annual AQI                       0
dtype: int64
Number of duplicate rows: 0


In [72]:
# Save the updated DataFrame to a new CSV file
output_file_path = '3.cleaned_airquality_data_stage3.csv'
aqi_full_pivot.to_csv(output_file_path, index=False)